### __LIBRARIES IMPORTATION__

In [ ]:
import fastf1 as ff1
import numpy as np
import pandas as pd
import pickle
import os

### __GENERAL DATA__

In [ ]:
year = 2022
circuit = 'Bahrein'
test_number = 2
session = 1

#### __STEP 0:__ Data Generation and pkl creation

In [ ]:
# ff1.Cache.enable_cache('C:/Users/migue/Documents/F1 Data Center/Datos/fastf1_Cache')
# test = ff1.get_testing_session(2023,1,2)
# test.load()

#### __STEP 1:__ Data Transformation into Pandas DF

In [ ]:
path = path =  f'C:\\Users\\migue\\Documents\\F1 Data Center\\Datos\\fastf1_Testing_Timing_Data\\{year}\\timing_data_t{test_number}_s{session}.ff1pkl'
with open(path, 'rb') as f:
    data = pickle.load(f)

dataTuple = data['data'][0]

df = pd.DataFrame(dataTuple)

print(df.shape)

#Eliminar Datos Corrompidos
nanDF = df.isna()
df['TotalNan'] = nanDF.sum(axis=1) 

df = df.loc[(df['TotalNan'] <= 2), :]
df = df.drop(['TotalNan'], axis=1)
df = df.reset_index(drop=True)

#BUGS
df = df.loc[df['Driver'].astype(int) < 100, :]

colNames: list = df.columns.values.tolist() #Column names
print(colNames)

In [ ]:
display(df)

#### __STEP 2:__ Pandas Modification(Data Transformation, Nan Values, remove unnecessary Columns...)

In [ ]:
# Unnecessary Columns Removement
df = df.drop(['PitOutTime', 'PitInTime', 'Sector1SessionTime','Sector2SessionTime','Sector3SessionTime', 'SpeedFL','IsPersonalBest'], axis=1)
df.head()

In [ ]:
import sys
path = "/Users/migue/Documents/F1 Data Center/F1-Data-Center/Programs/Python/"
sys.path.append(path)
import multidata

# Transform Driver Number into Drivers Name
def transformNum(row, year = year):
    numb = row['Driver'] 
    return multidata.numberToNames(year, numb).name

df['Driver'] = df.apply(transformNum, axis=1)
# display(df)

In [ ]:
#Transform time instances into miliseconds except 'Time' which will be transformed into minutes

def toMins(row):
    try: 
        return row['Time'].total_seconds() / 60
    except:
        return np.nan

def toMiliscs(elem):
    try: 
        return elem.total_seconds() *1000
    except:
        return np.nan

df['Time'] = df.apply(toMins, axis=1)
df['LapTime'] = df['LapTime'].apply(toMiliscs)
df['Sector1Time'] = df['Sector1Time'].apply(toMiliscs)
df['Sector2Time'] = df['Sector2Time'].apply(toMiliscs)
df['Sector3Time'] = df['Sector3Time'].apply(toMiliscs)
display(df)

In [ ]:
# path = "C:\\Users\\migue\\Documents\\F1 Data Center\\Datos\\PreSeasonTesting\\2022\\Test2_Ses1.csv"
# df.to_csv(path, index=False)

In [ ]:
colsName = ['Driver', 'Team', 'Year', 'Circuit', 'Test_number', 'Session', 'Total_PitStops', 'Total_Laps', 'Fastest_Lap', 'Qualy_Times', 'Total_Qualy_Laps', 'Avg_Qualy_Track_Time', 
'Avg_Qualy_Speed1', 'Avg_Qualy_Speed2', 'Avg_Qualy_SpeedST', 'Sector1_Qualy', 'Total_S1Q_Laps', 'Sector2_Qualy', 'Total_S2Q_Laps', 'Sector3_Qualy', 'Total_S3Q_Laps', 
'Race_Times', 'Total_Race_Laps', 'Avg_Race_Track_Time', 'Avg_Race_Speed1', 'Avg_Race_Speed2', 'Avg_Race_SpeedST', 'Sector1_Race', 'Total_S1R_Laps', 'Sector2_Race', 'Total_S2R_Laps',
'Sector3_Race','Total_S3R_Laps', 'FinalSeasonResult']

drivers = df['Driver'].unique()
rows = len(drivers)
data = np.empty((rows, len(colsName)),  dtype='object')

In [ ]:
def returnQualy_RaceLaps(times: np.array, mode: str = 'tt'):
    qualyLaps = []
    qualyIndeces = []
    raceLaps = []
    raceIndices = []
    minTime = np.nanmin(times)
    qualyTop = (minTime * 102)/100
    raceTop = (minTime * 110)/100
    for t,time in enumerate(times):
        if time != np.nan:
            if time <= qualyTop:
                qualyLaps.append(time)
                qualyIndeces.append(t)
            elif time <= raceTop:
                raceLaps.append(time)
                raceIndices.append(t)

    if mode == 'tt':
        if len(qualyLaps) == 0 and len(raceLaps) == 0:
            return [0], [0], [0], [0]
        elif len(raceLaps) == 0:
            return np.array(qualyLaps), qualyIndeces, [0], [0]
        else:
            return np.array(qualyLaps), qualyIndeces, np.array(raceLaps), raceIndices
    else: 
        if len(qualyLaps) == 0 and len(raceLaps) == 0:
            return [0], [0]
        elif len(raceLaps) == 0:
            return np.array(qualyLaps), [0]
        else:
            return np.array(qualyLaps), np.array(raceLaps)

In [ ]:
teams = multidata.f1_teams(drivers, year, 'Default', 'Default').equipos
teams = multidata.lastTeamName(teams).correct
for d,driv in enumerate(drivers):
    data[d,colsName.index('Driver')] = driv
    data[d,colsName.index('Team')] = teams[d]
    data[d,colsName.index('Year')] = year
    data[d,colsName.index('Circuit')] = circuit
    data[d,colsName.index('Test_number')] = test_number
    data[d,colsName.index('Session')] = session

    dfDriv = df.loc[(df['Driver'] == driv), :] 
    data[d,colsName.index('Total_PitStops')] = dfDriv['NumberOfPitStops'].max()
    data[d,colsName.index('Total_Laps')] = dfDriv['NumberOfLaps'].max()

    allTimes = dfDriv['LapTime'].to_numpy()
    qualyLaps, qualyIndeces, raceLaps, raceIndices = returnQualy_RaceLaps(allTimes)

    data[d,colsName.index('Qualy_Times')] = int(np.nanmean(qualyLaps, dtype=np.float64))
    data[d,colsName.index('Total_Qualy_Laps')] = len(qualyLaps)
    dfDrivQuali = dfDriv.iloc[qualyIndeces, :]
    data[d,colsName.index('Avg_Qualy_Track_Time')] = int(np.nanmean(dfDrivQuali['Time'].to_numpy()))
    data[d,colsName.index('Avg_Qualy_Speed1')] = int(np.nanmean(dfDrivQuali['SpeedI1'].to_numpy()))
    data[d,colsName.index('Avg_Qualy_Speed2')] = int(np.nanmean(dfDrivQuali['SpeedI2'].to_numpy()))
    data[d,colsName.index('Avg_Qualy_SpeedST')] = int(np.nanmean(dfDrivQuali['SpeedST'].to_numpy()))

    data[d,colsName.index('Race_Times')] = int(np.mean(raceLaps, dtype=np.float64))
    data[d,colsName.index('Total_Race_Laps')] = len(raceLaps)
    dfDrivrace = dfDriv.iloc[raceIndices, :]
    data[d,colsName.index('Avg_Race_Track_Time')] = int(np.nanmean(dfDrivrace['Time'].to_numpy()))
    data[d,colsName.index('Avg_Race_Speed1')] = int(np.nanmean(dfDrivrace['SpeedI1'].to_numpy()))
    data[d,colsName.index('Avg_Race_Speed2')] = int(np.nanmean(dfDrivrace['SpeedI2'].to_numpy()))
    data[d,colsName.index('Avg_Race_SpeedST')] = int(np.nanmean(dfDrivrace['SpeedST'].to_numpy()))

    s = dfDriv['Sector1Time'].to_numpy()
    qualyLaps, raceLaps = returnQualy_RaceLaps(s, mode='s')

    data[d,colsName.index('Sector1_Qualy')] = int(np.mean(qualyLaps))
    data[d,colsName.index('Total_S1Q_Laps')] = len(qualyLaps)
    data[d,colsName.index('Sector1_Race')] = int(np.mean(raceLaps))
    data[d,colsName.index('Total_S1R_Laps')] = len(raceLaps)

    s = dfDriv['Sector2Time'].to_numpy()
    qualyLaps, raceLaps = returnQualy_RaceLaps(s, mode='s')

    data[d,colsName.index('Sector2_Qualy')] = int(np.mean(qualyLaps))
    data[d,colsName.index('Total_S2Q_Laps')] = len(qualyLaps)
    data[d,colsName.index('Sector2_Race')] = int(np.mean(raceLaps))
    data[d,colsName.index('Total_S2R_Laps')] = len(raceLaps)

    s = dfDriv['Sector3Time'].to_numpy()
    qualyLaps, raceLaps = returnQualy_RaceLaps(s, mode='s')

    data[d,colsName.index('Sector3_Qualy')] = int(np.mean(qualyLaps))
    data[d,colsName.index('Total_S3Q_Laps')] = len(qualyLaps)
    data[d,colsName.index('Sector3_Race')] = int(np.mean(raceLaps))
    data[d,colsName.index('Total_S3R_Laps')] = len(raceLaps)

    data[d,colsName.index('FinalSeasonResult')] = multidata.seasonResult(year,[driv]).finalResult[0]

In [ ]:
# import csv
# path =  f'C:\\Users\\migue\\Documents\\F1 Data Center\\Datos\\fastf1_Testing_Timing_Data\\{year}'
# os.chdir(path)

# with open('Test1_Ses1.csv','w+', newline= '') as file:
#     writer = csv.writer(file,delimiter=';')
#     writer.writerow(colsName)
#     writer.writerows(data)


In [ ]:
# dfDriv = df.loc[(df['Driver'] == 'Alexander ALBON'), :]
# dfDriv['NumberOfLaps'].max()

# display(dfDriv)